In [45]:
# Tensorflow / Keras
import tensorflow as tf
from tensorflow import keras 
from keras.models import Sequential # creating a linear stack of NN
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout 

# data manipulation
import pandas as pd
import numpy as np

# sklearn 
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder

# visualisation 
import cv2 
import matplotlib
import matplotlib.pyplot as plt



In [46]:
import sys
import os

# assigning main directory path to a variable
main_dir = os.path.dirname(sys.path[0])
print(main_dir)

d:\Coding


In [47]:
# location prefix of images
IMG_LOCATION = main_dir + "\\Caltech-101-CNN\\data\\101_ObjectCategories\\"

#categories we are interested in 
LABELS = set(['dalmatian','hedgehog','llama', 'panda'])

# create two lists to contain image paths and image labels
ImagePaths = []
ListLabels = []
for label in LABELS:
    for image in list(os.listdir(IMG_LOCATION + label)):
        ImagePaths = ImagePaths + [IMG_LOCATION + label + "/" + image]
        ListLabels = ListLabels + [label]

# load images and resizing them
data = []
for img in ImagePaths:
    image = cv2.imread(img)
    image = cv2.resize(image, (128,128))
    data.append(image)

# convert image data to numpy array and standardize values 
data = np.array(data, dtype = "float") / 255

The shape of data is 4 dimensional `(237, 128, 128, 3)`
- 237 : objects/images
- 128 : pixels along x axis
- 128 : pixels along y axis
- 3 : the colours R, G, B channels

That is `Sample, rows, columns, channels`

In [48]:
data.shape

(237, 128, 128, 3)

Our target will be one of the four values - 
- llama
- panda
- dalmatian
- hedgehog
so we create a np array and encode it too!

In [49]:
# also converting labels list to np array 
LablesArray = np.array(ListLabels)

# Encode labels 
enc = OrdinalEncoder()
y = enc.fit_transform(LablesArray.reshape(-1,1))

Now we need to create training and testing samples using sklearn!


In [53]:
X_train, X_test, y_train, y_test = train_test_split(data,y, test_size = 0.2, random_state= 42)

# Print shapes
print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_test: ", y_test.shape)

Shape of X_train:  (189, 128, 128, 3)
Shape of y_train:  (189, 1)
Shape of X_test:  (48, 128, 128, 3)
Shape of y_test:  (48, 1)
